In [3]:
from langchain.prompts import PromptTemplate

# Prompt for extracting required skills
skills_prompt_template = """
You are an expert curriculum designer. A user with the following background:
{person_info}
wants to accomplish the following goal:
{goal}

1. Identify the core set of skills or knowledge areas required to achieve this goal.
2. Break these skills into logical subskills if applicable.
3. Provide a short explanation of why each skill is necessary.

Present the result in a structured JSON format with keys: 'skill', 'subskills', and 'rationale'.
"""

skills_prompt = PromptTemplate(
    input_variables=["person_info", "goal"],
    template=skills_prompt_template
)

# Prompt for resource recommendations
resources_prompt_template = """
You are an expert in recommending learning resources. Given the following skill:
{skill_name}

Suggest 3 to 5 high-quality resources that are beginner-friendly and relevant to mastering this skill. 
For each resource, provide:
- resource_title
- resource_link
- short_description

Return as a JSON list of resource objects.
"""

resources_prompt = PromptTemplate(
    input_variables=["skill_name"],
    template=resources_prompt_template
)


In [2]:
import os
from langchain_community.llms import OpenAI
from langchain.chains import LLMChain

# Initialize the LLM (here we use OpenAI’s model)
llm = OpenAI(
    temperature=0.7, 
    model_name="gpt-3.5-turbo-instruct",  # or "gpt-3.5-turbo" if using Chat model
)

# Create the chain for extracting skills
skills_chain = LLMChain(
    llm=llm,
    prompt=skills_prompt
)

# Create the chain for resource recommendations
resources_chain = LLMChain(
    llm=llm,
    prompt=resources_prompt
)


ModuleNotFoundError: No module named 'langchain_community'

In [7]:
# Example user info
person_info = """
- Currently knows basic HTML.
- Some background in CSS but never used frameworks.
- Comfortable with simple JavaScript.
"""
goal = "I want to build a modern, responsive website with user authentication and a backend."

# Call the skills extraction chain
skills_output = skills_chain.run({
    "person_info": person_info,
    "goal": goal
})

print("SKILLS OUTPUT (JSON):")
print(skills_output)


NameError: name 'skills_chain' is not defined

In [8]:
import json

parsed_skills = json.loads(skills_output)
skills_data = parsed_skills.get("skills", [])

full_plan = []

for skill_item in skills_data:
    skill_name = skill_item["skill"]
    
    resources_output = resources_chain.run({"skill_name": skill_name})
    try:
        resources_recommended = json.loads(resources_output)
    except:
        resources_recommended = [{"error": "Could not parse resources"}]
    
    # Combine skill info and resource recommendations
    plan_entry = {
        "skill": skill_name,
        "subskills": skill_item.get("subskills", []),
        "rationale": skill_item.get("rationale", ""),
        "resources": resources_recommended
    }
    full_plan.append(plan_entry)

print("FULL PLAN:")
print(json.dumps(full_plan, indent=2))


NameError: name 'skills_output' is not defined